In [3]:
import json
import websocket
import threading
import sqlite3
import requests

def get_top_50_pairs():
    response = requests.get('https://api.binance.com/api/v3/ticker/24hr')
    tickers = response.json()
    
    # Sort by 24-hour quote volume
    sorted_tickers = sorted(tickers, key=lambda x: float(x.get('quoteVolume', 0)), reverse=True)
    
    return [ticker['symbol'] for ticker in sorted_tickers[:50]]

def on_message(ws, message):
    data = json.loads(message)
    kline = data['k']

    # Extract OHLCV data
    open_price = kline['o']
    high_price = kline['h']
    low_price = kline['l']
    close_price = kline['c']
    volume = kline['v']
    timestamp = kline['t']  # You might want to convert this to a human-readable date
    symbol = data['s']

    # Connect to the SQLite database
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    print("Inserting ", open_price, "for ", symbol, " at ", timestamp)
    print("---------------------------------------------------------")
    # Insert the data into the ohlcv table
    c.execute("INSERT INTO ohlcv (date, symbol, open, high, low, close, volume) VALUES (?, ?, ?, ?, ?, ?, ?)",
              (timestamp, symbol, open_price, high_price, low_price, close_price, volume))

    conn.commit()
    conn.close()

In [4]:
def connect_to_websocket(symbol):
    ws = websocket.WebSocketApp(f"wss://stream.binance.com:9443/ws/{symbol}@kline_1m",
                                on_message=on_message)
    ws.run_forever()

def threaded_websocket_connection(symbol):
    thread = threading.Thread(target=connect_to_websocket, args=(symbol,))
    thread.start()

In [5]:
top_50_pairs = get_top_50_pairs()
print(top_50_pairs)
for pair in top_50_pairs:
    threaded_websocket_connection(pair)

{'symbol': 'ETHBTC', 'status': 'TRADING', 'baseAsset': 'ETH', 'baseAssetPrecision': 8, 'quoteAsset': 'BTC', 'quotePrecision': 8, 'quoteAssetPrecision': 8, 'baseCommissionPrecision': 8, 'quoteCommissionPrecision': 8, 'orderTypes': ['LIMIT', 'LIMIT_MAKER', 'MARKET', 'STOP_LOSS_LIMIT', 'TAKE_PROFIT_LIMIT'], 'icebergAllowed': True, 'ocoAllowed': True, 'quoteOrderQtyMarketAllowed': True, 'allowTrailingStop': True, 'cancelReplaceAllowed': True, 'isSpotTradingAllowed': True, 'isMarginTradingAllowed': True, 'filters': [{'filterType': 'PRICE_FILTER', 'minPrice': '0.00001000', 'maxPrice': '922327.00000000', 'tickSize': '0.00001000'}, {'filterType': 'LOT_SIZE', 'minQty': '0.00010000', 'maxQty': '100000.00000000', 'stepSize': '0.00010000'}, {'filterType': 'ICEBERG_PARTS', 'limit': 10}, {'filterType': 'MARKET_LOT_SIZE', 'minQty': '0.00000000', 'maxQty': '5669.98621083', 'stepSize': '0.00000000'}, {'filterType': 'TRAILING_DELTA', 'minTrailingAboveDelta': 10, 'maxTrailingAboveDelta': 2000, 'minTraili

KeyError: 'quoteVolume'